Assignment 2 : clustering neighbourhoods in Toronto

Import required packages

In [86]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request


create BS object, list and create dictionary

In [87]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

#create soup
soup = BeautifulSoup(source)

#create List
list_postalcode = [1,2,3]

#create dictionary called cell having 3 keys PostalCode, Borough, Neighbourhood
cell = {"PostalCode" :"","Borough" :"","Neighbourhood" : ""}

In [88]:
#create table
table_contents=[]
#create soup, get table
table=soup.find('table')
#create dict with 3 keys, extract data and append to table
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
#create dataframe using list table_contents
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#print(df.shape)
#print(type(df))


In [89]:
#attempt to use geocoder, didnt work
import geocoder
g = geocoder.canadapost('M3A, Toronto, Ontario'.format(postal_code))
print(g.latlng)

['', '']


In [90]:
import geocoder # import geocoder
import pgeocode #utilize pgecode instead

In [91]:
#this cell utilizes pgeocode to get lats and logs into lists of lats and longs
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = df['PostalCode'].tolist()
latitudes = []
longitudes = []
for i, postal_code in enumerate(postal_codes):
    # initialize your variable to None
    #print(f'--Getting Postal Code: {postal_code}')
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        #print(f'Postal Code {postal_code} has been retrieved. {len(postal_codes) - (i + 1)} codes left')
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

In [92]:
#this cell inserts the lat and longitudes into the main dataframe
df= pd.DataFrame(table_contents)
df1 = pd.DataFrame(latitudes)
df.insert(3,"Latitude",latitudes)
df.insert(4,"Longitude",longitudes)
print(df)



    PostalCode                                            Borough  \
0          M3A                                         North York   
1          M4A                                         North York   
2          M5A                                   Downtown Toronto   
3          M6A                                         North York   
4          M7A                                       Queen's Park   
..         ...                                                ...   
98         M8X                                          Etobicoke   
99         M4Y                                   Downtown Toronto   
100        M7Y  East TorontoBusiness reply mail Processing Cen...   
101        M8Y                                          Etobicoke   
102        M8Z                                          Etobicoke   

                                          Neighborhood  Latitude  Longitude  
0                                            Parkwoods   43.7545   -79.3300  
1              